In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
traindata=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
testdata=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [85]:
xtrain=traindata.drop('SalePrice',axis=1)
ytrain=traindata['SalePrice']
xtrain=xtrain.drop(['Alley','PoolQC','Fence','MiscFeature','FireplaceQu'],axis=1)
trainmean=xtrain.mean()
trainmode=xtrain.mode()
xtrain=xtrain.fillna(trainmean)
xtrain=xtrain.fillna(trainmode.iloc[0])


for col in xtrain.select_dtypes('object') :
    xtrain[col]=xtrain[col].astype('category').cat.codes
xtrain['MSSubClass']=xtrain['MSSubClass'].astype('category').cat.codes
xtrain['OverallQual']=xtrain['OverallQual'].astype('category').cat.codes
xtrain['OverallCond']=xtrain['OverallCond'].astype('category').cat.codes


#xtrain=
# MSSubClass 16 OverallQual 10 OverallCond 10
xtrainuse=xtrain[200:]
ytrainuse=ytrain[200:]
xvaild=xtrain[:200]
yvaild=ytrain[:200]

In [73]:
xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   float64
 1   MSSubClass     1460 non-null   float64
 2   MSZoning       1460 non-null   float64
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   float64
 5   Street         1460 non-null   float64
 6   LotShape       1460 non-null   float64
 7   LandContour    1460 non-null   float64
 8   Utilities      1460 non-null   float64
 9   LotConfig      1460 non-null   float64
 10  LandSlope      1460 non-null   float64
 11  Neighborhood   1460 non-null   float64
 12  Condition1     1460 non-null   float64
 13  Condition2     1460 non-null   float64
 14  BldgType       1460 non-null   float64
 15  HouseStyle     1460 non-null   float64
 16  OverallQual    1460 non-null   float64
 17  OverallCond    1460 non-null   float64
 18  YearBuil

In [72]:
xtrain

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.000000,0.357143,0.75,0.150685,0.033420,1.0,1.0,1.0,0.0,1.0,...,0.111517,0.000000,0.0,0.0,0.0,0.00000,0.090909,0.50,1.0,0.8
1,0.000685,0.000000,0.75,0.202055,0.038795,1.0,1.0,1.0,0.0,0.5,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.363636,0.25,1.0,0.8
2,0.001371,0.357143,0.75,0.160959,0.046507,1.0,0.0,1.0,0.0,1.0,...,0.076782,0.000000,0.0,0.0,0.0,0.00000,0.727273,0.50,1.0,0.8
3,0.002056,0.428571,0.75,0.133562,0.038561,1.0,0.0,1.0,0.0,0.0,...,0.063985,0.492754,0.0,0.0,0.0,0.00000,0.090909,0.00,1.0,0.0
4,0.002742,0.357143,0.75,0.215753,0.060576,1.0,0.0,1.0,0.0,0.5,...,0.153565,0.000000,0.0,0.0,0.0,0.00000,1.000000,0.50,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.997258,0.357143,0.75,0.140411,0.030929,1.0,1.0,1.0,0.0,1.0,...,0.073126,0.000000,0.0,0.0,0.0,0.00000,0.636364,0.25,1.0,0.8
1456,0.997944,0.000000,0.75,0.219178,0.055505,1.0,1.0,1.0,0.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.090909,1.00,1.0,0.8
1457,0.998629,0.428571,0.75,0.154110,0.036187,1.0,1.0,1.0,0.0,1.0,...,0.109689,0.000000,0.0,0.0,0.0,0.16129,0.363636,1.00,1.0,0.8
1458,0.999315,0.000000,0.75,0.160959,0.039342,1.0,1.0,1.0,0.0,1.0,...,0.000000,0.202899,0.0,0.0,0.0,0.00000,0.272727,1.00,1.0,0.8


In [91]:
model=keras.models.Sequential()
model.add(keras.layers.Dense(100,input_shape=(75,),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(50,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(30,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))
model.compile(loss='mse',optimizer='Adam')


In [92]:
model.fit(xtrainuse,ytrainuse,epochs=100,validation_data=(xvaild,yvaild))

Epoch 1/100
40/40 [==============================] - 1s 7ms/step - loss: 39169202625.5610 - val_loss: 29943363584.0000
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 23437532184.9756 - val_loss: 4079931904.0000
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 9111042634.9268 - val_loss: 4867280384.0000
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 8257902454.6341 - val_loss: 3634134272.0000
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 8128014585.7561 - val_loss: 3297989376.0000
Epoch 6/100
40/40 [==============================] - 0s 3ms/step - loss: 7392491095.4146 - val_loss: 3667976448.0000
Epoch 7/100
40/40 [==============================] - 0s 3ms/step - loss: 6400860222.4390 - val_loss: 2993143808.0000
Epoch 8/100
40/40 [==============================] - 0s 3ms/step - loss: 6239453296.3902 - val_loss: 2521255168.0000
Epoch 9/100
40/40 [==============================] - 0s 3ms/s

In [78]:
testdata

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
